In [16]:
%matplotlib qt

import numpy as np
import pandas as pd
import csv


import SimpleITK as sitk
from radiomics import featureextractor
import multiprocessing
from multiprocessing import Pool
from itertools import repeat
from tqdm import tqdm

In [2]:
def flatten_mask_labels(mask_path):
    ma = sitk.ReadImage(mask_path)
    ma_arr = sitk.GetArrayFromImage(ma)
    for l in range(1,ma_arr.max()+1):
        ma_arr[ma_arr == l] = 1

    ma_merged = sitk.GetImageFromArray(ma_arr)
    ma_merged.CopyInformation(ma)
    
    return ma_merged


In [3]:
wd = os.getcwd()
root = wd[:-8]

main_dir = root+'0.0 Main'
input_dir = root+'0.1 inputs'
output_dir = root+'0.2 outputs'
params_dir = root+'0.3 parameters'

In [4]:
path_dicts = []
with open(input_dir+'\CT_paths.csv', 'r') as file:
    csv_file = csv.DictReader(file)
    for row in csv_file:
        path_dicts.append(row)
    n_instances = len(path_dicts)
    print(f'Parsed {n_instances} image-mask pairs')

Parsed 74 image-mask pairs


In [5]:
parameter_file = params_dir+'\\Params.yaml'
extractor = featureextractor.RadiomicsFeatureExtractor(parameter_file)

In [6]:
def extract_feats(image_mask_pair,params):
    image_path = image_mask_pair['Image']
    mask_path = image_mask_pair['Mask']
    
    flat_mask = flatten_mask_labels(mask_path)  # Feature extraction will only occur for masks with label==1, this function takes masks with multiple labels and sets them to 1
    extractor = featureextractor.RadiomicsFeatureExtractor(params)
    features = extractor.execute(image_path, flat_mask, label=1)
    
    return features


In [9]:
result =[]
for i in tqdm(range(n_instances)):
    result.append(extract_feats(path_dicts[i],params=parameter_file))

100%|██████████| 74/74 [09:16<00:00,  7.53s/it]


In [8]:
# %%time
# workers = multiprocessing.cpu_count()
# with Pool(2) as p:
#     M = p.map(extract_feats, path_dicts[:2])

In [26]:
df_paths = pd.DataFrame(path_dicts)
df_result = pd.DataFrame(result)

In [27]:
merged = pd.concat([df_paths, df_result], axis=1)
merged

,Image,Mask,Class,diagnostics_Versions_PyRadiomics,diagnostics_Versions_Numpy,diagnostics_Versions_SimpleITK,diagnostics_Versions_PyWavelet,diagnostics_Versions_Python,diagnostics_Configuration_Settings,diagnostics_Configuration_EnabledImageTypes,...,original_gldm_GrayLevelNonUniformity,original_gldm_GrayLevelVariance,original_gldm_HighGrayLevelEmphasis,original_gldm_LargeDependenceEmphasis,original_gldm_LargeDependenceHighGrayLevelEmphasis,original_gldm_LargeDependenceLowGrayLevelEmphasis,original_gldm_LowGrayLevelEmphasis,original_gldm_SmallDependenceEmphasis,original_gldm_SmallDependenceHighGrayLevelEmphasis,original_gldm_SmallDependenceLowGrayLevelEmphasis
0,X:\0.0 DATASETS\QIN DATASET\QIN-HEADNECK\0.5 A...,X:\0.0 DATASETS\QIN DATASET\QIN-HEADNECK\0.5 A...,0,v3.0.1,1.19.5,2.0.1,1.1.1,3.8.6,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},...,1247.3180764774045,29.058511169980967,1791.8744689069138,65.28988026264967,116873.93781382774,0.05128055864376676,0.0014728980388839933,0.11973553620222611,197.00956728852228,0.0003111216965486707
1,X:\0.0 DATASETS\QIN DATASET\QIN-HEADNECK\0.5 A...,X:\0.0 DATASETS\QIN DATASET\QIN-HEADNECK\0.5 A...,0,v3.0.1,1.19.5,2.0.1,1.1.1,3.8.6,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},...,3848.2292576419213,50.74631194430316,1643.833437305053,105.25483468496569,185223.83977957995,0.12650342007809565,0.002687990672060076,0.12416956683156527,151.4897184606033,0.0005755680993951987
2,X:\0.0 DATASETS\QIN DATASET\QIN-HEADNECK\0.5 A...,X:\0.0 DATASETS\QIN DATASET\QIN-HEADNECK\0.5 A...,0,v3.0.1,1.19.5,2.0.1,1.1.1,3.8.6,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},...,1435.314659685864,0.6039068967407691,19.13219895287958,140.94293193717277,2739.3641361256546,7.885583964039368,0.06269395397775167,0.03079315866109682,0.5839748682367925,0.002791032205941674
3,X:\0.0 DATASETS\QIN DATASET\QIN-HEADNECK\0.5 A...,X:\0.0 DATASETS\QIN DATASET\QIN-HEADNECK\0.5 A...,0,v3.0.1,1.19.5,2.0.1,1.1.1,3.8.6,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},...,2091.987142397943,26.42307293666949,1877.6421568627452,96.29974284795885,173625.0013661202,0.08360825174410205,0.0017883644279432,0.11621774549518847,235.90467092951812,0.0002590427799270595
4,X:\0.0 DATASETS\QIN DATASET\QIN-HEADNECK\0.5 A...,X:\0.0 DATASETS\QIN DATASET\QIN-HEADNECK\0.5 A...,1,v3.0.1,1.19.5,2.0.1,1.1.1,3.8.6,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},...,529.3063636363636,21.44118161157025,1550.9977272727272,93.72181818181818,151369.97318181818,0.10756450625155124,0.0033672388981178195,0.11476433406346209,141.06250303149494,0.0005385302476285819
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,X:\0.0 DATASETS\QIN DATASET\QIN-HEADNECK\0.5 A...,X:\0.0 DATASETS\QIN DATASET\QIN-HEADNECK\0.5 A...,0,v3.0.1,1.19.5,2.0.1,1.1.1,3.8.6,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},...,6173.188913982329,48.163956804495434,1781.8029240461192,140.6980862950196,258288.15539442925,1.738918837456069,0.011003244493782602,0.08074797741180433,111.0416454399007,0.0007152844571251441
70,X:\0.0 DATASETS\QIN DATASET\QIN-HEADNECK\0.5 A...,X:\0.0 DATASETS\QIN DATASET\QIN-HEADNECK\0.5 A...,0,v3.0.1,1.19.5,2.0.1,1.1.1,3.8.6,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},...,1818.5507370831544,56.81265823898475,1792.2499642192643,97.18677544010305,179433.55524545585,0.1172763906785908,0.00263295036950073,0.14584015928708327,243.4437922263317,0.0005463974411961242
71,X:\0.0 DATASETS\QIN DATASET\QIN-HEADNECK\0.5 A...,X:\0.0 DATASETS\QIN DATASET\QIN-HEADNECK\0.5 A...,0,v3.0.1,1.19.5,2.0.1,1.1.1,3.8.6,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},...,3517.5667391304346,1.6914780718336484,783.4773913043479,179.46347826086958,142216.98847826087,0.2270082658331856,0.0014432830947360299,0.034450737706023686,24.42163661937633,0.00020894281241149614
72,X:\0.0 DATASETS\